In [1]:
from tqdm import tqdm
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

/var/data/python/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
df_anomaly = pd.read_csv('/home/thetom205/traktor hack/data/для анализа/dataset._anomaly.csv',sep=';')
df_anomaly.replace('        -', np.nan, inplace=True)
df_anomaly.replace('-', np.nan, inplace=True)

/tmp/ipykernel_446/1735381011.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_anomaly.replace('        -', np.nan, inplace=True)
/tmp/ipykernel_446/1735381011.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_anomaly.replace('-', np.nan, inplace=True)


In [3]:
df_problems = pd.read_csv('/home/thetom205/traktor hack/data/для анализа/dataset._problems.csv',sep=';')
df_problems.replace('        -', np.nan, inplace=True)
df_problems.replace('-', np.nan, inplace=True)

/tmp/ipykernel_446/3875790439.py:1: DtypeWarning: Columns (3,11,12,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,37,41,42,44) have mixed types. Specify dtype option on import or set low_memory=False.
  df_problems = pd.read_csv('/home/thetom205/traktor hack/data/для анализа/dataset._problems.csv',sep=';')
/tmp/ipykernel_446/3875790439.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_problems.replace('        -', np.nan, inplace=True)
/tmp/ipykernel_446/3875790439.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
 

In [4]:
df_normal = pd.read_csv('/home/thetom205/traktor hack/data/для анализа/dataset._normal.csv',sep=';')
df_normal.replace('        -', np.nan, inplace=True)
df_normal.replace('-', np.nan, inplace=True)

/tmp/ipykernel_446/4066241437.py:1: DtypeWarning: Columns (3,12,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,37,41,42,44) have mixed types. Specify dtype option on import or set low_memory=False.
  df_normal = pd.read_csv('/home/thetom205/traktor hack/data/для анализа/dataset._normal.csv',sep=';')
/tmp/ipykernel_446/4066241437.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_normal.replace('        -', np.nan, inplace=True)
/tmp/ipykernel_446/4066241437.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_norma

Метка 0 - нормальное состояние, 1 - аномалия, 2 - поломка

In [5]:
df_normal['target'] = 0
df_anomaly['target'] = 1
df_problems['target'] = 2

In [6]:
train = pd.concat([
    df_normal,
    df_anomaly,
    df_problems
])

In [7]:
column_names_train = train.columns.tolist()
for i in train:
  if (train[i].isnull().sum() / len(train[i])) * 100 > 90:
    del train[i]

In [8]:
train.dropna(thresh=5, inplace=True, axis=0)

In [9]:
for i in train.select_dtypes(include=['object']):
   train[i] = train[i].fillna(train[i].mode().iloc[0])

In [10]:
def make_harmonic_features(value, period=24):
    value *= 2 * np.pi / period
    return np.cos(value), np.sin(value)

In [11]:
train['Темп.масла двиг.,°С'] = train['Темп.масла двиг.,°С'].str.replace(',', '.').astype('float')
train['Обор.двиг.,об/мин'] = train['Обор.двиг.,об/мин'].str.replace(',', '.').astype('float')
train['Полож.пед.акселер.,%'] = train['Полож.пед.акселер.,%'].str.replace(',', '.').astype('float')

In [12]:
train_RUL = train[['Засоренность фильтра рулевого управления (spn3844)','target']]
train_RUL=train_RUL.astype('float64')

In [13]:
X, y = train_RUL.drop(['target'], axis=1), train_RUL.target

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)

In [15]:
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, stratify=y_test, test_size=0.3, random_state=42)

In [16]:
# model = CatBoostClassifier(
#     iterations=200, depth=10, learning_rate=0.1,
#    loss_function='MultiClass', verbose=100)
# model.fit(X_train, y_train,eval_set=(X_val,y_val),plot=True)

In [18]:
from joblib import load

model = load("model_rulev.pkl")

In [19]:
y_pred = model.predict(X_test)

In [20]:
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)

/var/data/python/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/var/data/python/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.8075073814231926
Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      0.25      0.40     98995
         1.0       0.00      0.00      0.00     39838
         2.0       0.80      1.00      0.89    451839

    accuracy                           0.81    590672
   macro avg       0.60      0.42      0.43    590672
weighted avg       0.78      0.81      0.75    590672



/var/data/python/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
y_pred_val = model.predict(X_val)

accuracy = accuracy_score(y_val, y_pred_val)
report = classification_report(y_val, y_pred_val)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)

/var/data/python/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/var/data/python/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.8073088257369265
Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      0.25      0.40     42427
         1.0       0.00      0.00      0.00     17073
         2.0       0.80      1.00      0.89    193646

    accuracy                           0.81    253146
   macro avg       0.60      0.42      0.43    253146
weighted avg       0.78      0.81      0.75    253146



/var/data/python/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
df_imp = pd.DataFrame({
    'name': X.columns,
    'imp': model.get_feature_importance()
}).sort_values('imp', ascending=False)

In [23]:
df_imp

,name,imp
0,Засоренность фильтра рулевого управления (spn3...,100.0


In [24]:
from joblib import dump

dump(model, "model_rulev.pkl")

['model_rulev.pkl']

In [25]:
y_proba_val = model.predict_proba(X_val)

In [26]:
df_fin = pd.DataFrame({
    'normal':y_proba_val[:, 0],
    'anomaly': y_proba_val[:, 1],
    'problem' : y_proba_val[:, 2]
})

In [27]:
df_fin['normal'] = df_fin['normal'].mean()
df_fin['anomaly'] = df_fin['anomaly'].mean()
df_fin['problem'] = df_fin['problem'].mean()

In [28]:
df_fin.drop_duplicates()

,normal,anomaly,problem
0,0.16787,0.067423,0.764707
